In [7]:
import torch

model_path = "debug/20230710162225/admm_reconstruct.pth"
dict = torch.load(model_path, map_location=torch.device('cpu'))
print("epoch",dict['epoch'])
for k, v in dict["state_dict"].items():
    print(k,v)

# import math
# def closest_power_of_two(n):
#     power = math.floor(math.log2(n)) + 1
#     result = 2 ** power
#     return result
# print(closest_power_of_two(480))

epoch 30
module.alpha tensor([ 1.8266e-03,  1.2314e-03,  1.2191e-04,  8.7428e-05,  1.1441e-04,
         2.3787e-06,  2.9161e-06,  2.9988e-06, -6.2392e-07,  1.0285e-04])
module.beta tensor([5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05,
        5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05])
module.gamma_l1 tensor([5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05,
        5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05])
module.gamma_tv tensor([4.7113e-04, 1.3749e-04, 2.9682e-05, 1.1816e-04, 5.9887e-06, 4.1135e-05,
        3.1892e-05, 1.2694e-04, 2.4980e-04, 1.4926e-04])
module.delta tensor([5.1619e-06, 2.2406e-05, 1.2867e-04, 1.3560e-04, 4.1065e-05, 3.4946e-06,
        1.0602e-06, 5.5412e-07, 8.9650e-07, 1.3925e-04])
module.s tensor([5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05,
        5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05])
module.lambda_l1 tensor([5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e-05, 5.0000e